In [55]:
import pandas as pd

In [56]:
# Read the Excel workbook
df = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls')

In [57]:
df.head()

,Food Environment Atlas data download
0,Notes about the Food Environment Atlas downloa...
1,This file contains multiple spreadsheets:
2,1. A variable list that includes metadata abo...
3,2. Spreadsheets that contain data for each of...
4,3. County and State-level supplemental data t...


In [58]:
# Load Access, Stores, Assistance, Insecurity, Local, Health, Restaurants, Socioeconomic data sheets as dataframes 

access = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'ACCESS')
stores = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'STORES')
assistance = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'ASSISTANCE')
insecurity = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'INSECURITY')
local = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'LOCAL')
health = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'HEALTH')
restaurants = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'RESTAURANTS')
socioeconomic = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'SOCIOECONOMIC')
prices_taxes = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'PRICES_TAXES')

In [60]:
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [61]:
restaurants.columns

Index(['FIPS', 'State', 'County', 'FFR09', 'FFR14', 'PCH_FFR_09_14',
       'FFRPTH09', 'FFRPTH14', 'PCH_FFRPTH_09_14', 'FSR09', 'FSR14',
       'PCH_FSR_09_14', 'FSRPTH09', 'FSRPTH14', 'PCH_FSRPTH_09_14',
       'PC_FFRSALES07', 'PC_FFRSALES12', 'PC_FSRSALES07', 'PC_FSRSALES12'],
      dtype='object')

In [62]:
restaurant_cols = ['FIPS', 'State', 'County', 'FFRPTH14', 'FSRPTH14']
stores_cols = ['FIPS', 'State', 'County', 'GROCPTH14', 'SUPERCPTH14', 'CONVSPTH14', 'SPECSPTH14', 'SNAPSPTH16']
access_cols = ['FIPS', 'State', 'County', 'PCT_LACCESS_POP15', 'PCT_LACCESS_LOWI15', 'PCT_LACCESS_HHNV15', 'PCT_LACCESS_SNAP15', 'PCT_LACCESS_CHILD15', 'PCT_LACCESS_SENIORS15', 'PCT_LACCESS_WHITE15', 'PCT_LACCESS_BLACK15', 'PCT_LACCESS_HISP15', 'PCT_LACCESS_NHASIAN15', 'PCT_LACCESS_NHNA15', 'PCT_LACCESS_NHPI15', 'PCT_LACCESS_MULTIR15']
assistance_cols = ['FIPS', 'State', 'County', 'PCT_SNAP16', 'PCT_NSLP15', 'PCT_SBP15', 'PCT_SFSP15', 'PCT_WIC15']
prices_taxes_cols = ['FIPS', 'State', 'County', 'SODATAX_STORES14', 'SODATAX_VENDM14', 'CHIPSTAX_STORES14', 'CHIPSTAX_VENDM14', 'FOOD_TAX14']
local_cols = ['FIPS', 'State', 'County', 'FMRKTPTH16', 'PCT_FMRKT_SNAP16', 'PCT_FMRKT_WIC16', 'PCT_FMRKT_WICCASH16', 'PCT_FMRKT_SFMNP16', 'PCT_FMRKT_CREDIT16', 'PCT_FMRKT_FRVEG16', 'PCT_FMRKT_ANMLPROD16', 'PCT_FMRKT_BAKED16', 'PCT_FMRKT_OTHERFOOD16', 'FOODHUB16']
health_cols = ['FIPS', 'State', 'County', 'PCT_DIABETES_ADULTS13', 'PCT_OBESE_ADULTS13', 'PCT_HSPA15', 'RECFACPTH14']
socioeconomic_cols = ['FIPS', 'State', 'County', 'PCT_NHWHITE10', 'PCT_NHBLACK10', 'PCT_HISP10', 'PCT_NHASIAN10', 'PCT_NHNA10', 'PCT_NHPI10', 'PCT_65OLDER10', 'PCT_18YOUNGER10', 'MEDHHINC15', 'POVRATE15', 'METRO13']


In [63]:
features_new = stores_cols + assistance_cols + local_cols + health_cols + restaurant_cols + socioeconomic_cols + prices_taxes_cols + access_cols
prediction_features = list(set(features_new) - set(['FIPS', 'State', 'County', 'PCT_DIABETES_ADULTS13', 'PCT_OBESE_ADULTS13']))
target = 'PCT_OBESE_ADULTS13'




In [64]:
restaurants.columns

Index(['FIPS', 'State', 'County', 'FFR09', 'FFR14', 'PCH_FFR_09_14',
       'FFRPTH09', 'FFRPTH14', 'PCH_FFRPTH_09_14', 'FSR09', 'FSR14',
       'PCH_FSR_09_14', 'FSRPTH09', 'FSRPTH14', 'PCH_FSRPTH_09_14',
       'PC_FFRSALES07', 'PC_FFRSALES12', 'PC_FSRSALES07', 'PC_FSRSALES12'],
      dtype='object')

In [65]:

dfs = [stores, assistance, insecurity, local, health, restaurants, socioeconomic, prices_taxes]
for df in dfs:
    df.drop(columns=['State', 'County'], axis=1, inplace=True)
    df.set_index('FIPS', inplace=True)



In [66]:
stores.head()

,GROC09,GROC14,PCH_GROC_09_14,GROCPTH09,GROCPTH14,PCH_GROCPTH_09_14,SUPERC09,SUPERC14,PCH_SUPERC_09_14,SUPERCPTH09,...,PCH_SNAPS_12_16,SNAPSPTH12,SNAPSPTH16,PCH_SNAPSPTH_12_16,WICS08,WICS12,PCH_WICS_08_12,WICSPTH08,WICSPTH12,PCH_WICSPTH_08_12
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,6,4,-33.333333,0.110834,0.072209,-34.849716,1,1,0.0,0.018472,...,12.694878,0.674004,0.760911,12.894172,6,5,-16.66667,0.119156,0.090067,-24.412460
1003,24,29,20.833333,0.133775,0.144920,8.331001,6,6,0.0,0.033444,...,43.192771,0.725055,0.949753,30.990390,25,27,8.00000,0.141875,0.141517,-0.252126
1005,5,5,0.000000,0.180786,0.185963,2.863838,0,1,NaN,0.000000,...,0.956938,1.280590,1.354387,5.762745,6,7,16.66667,0.201099,0.257344,27.968330
1007,6,5,-16.666667,0.261540,0.222163,-15.055985,1,1,0.0,0.043590,...,20.512821,0.719122,0.864874,20.267995,6,5,-16.66667,0.277919,0.221268,-20.383970
1009,6,6,0.000000,0.104637,0.103952,-0.654897,1,1,0.0,0.017440,...,23.903509,0.657144,0.815946,24.165470,10,6,-40.00000,0.173028,0.103760,-40.033200


In [67]:
access.set_index('FIPS', drop=False, inplace=True)

In [68]:
access.head()

,FIPS,State,County,LACCESS_POP10,LACCESS_POP15,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,LACCESS_LOWI10,LACCESS_LOWI15,...,LACCESS_HISP15,PCT_LACCESS_HISP15,LACCESS_NHASIAN15,PCT_LACCESS_NHASIAN15,LACCESS_NHNA15,PCT_LACCESS_NHNA15,LACCESS_NHPI15,PCT_LACCESS_NHPI15,LACCESS_MULTIR15,PCT_LACCESS_MULTIR15
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,1001,AL,Autauga,18428.439685,17496.693038,-5.056026,33.769657,32.062255,5344.427472,6543.676824,...,471.136164,0.863345,86.767975,0.159000,61.169869,0.112092,8.817961,0.016159,482.848633,0.884808
1003,1003,AL,Baldwin,35210.814078,30561.264430,-13.204891,19.318473,16.767489,9952.144027,9886.831137,...,1377.874834,0.755973,212.946378,0.116833,181.649648,0.099662,14.819634,0.008131,1127.696098,0.618712
1005,1005,AL,Barbour,5722.305602,6069.523628,6.067799,20.840972,22.105560,3135.676086,2948.790251,...,509.377525,1.855183,17.096410,0.062266,39.960527,0.145539,8.082376,0.029436,462.382655,1.684025
1007,1007,AL,Bibb,1044.867327,969.378841,-7.224696,4.559753,4.230324,491.449066,596.162829,...,8.596762,0.037516,1.994318,0.008703,2.513097,0.010967,0.000000,0.000000,5.259244,0.022951
1009,1009,AL,Blount,1548.175559,3724.428242,140.568857,2.700840,6.497380,609.027708,1650.959482,...,497.489891,0.867886,8.428994,0.014705,28.938242,0.050484,1.062851,0.001854,202.914187,0.353990


In [69]:
stores.head()

,GROC09,GROC14,PCH_GROC_09_14,GROCPTH09,GROCPTH14,PCH_GROCPTH_09_14,SUPERC09,SUPERC14,PCH_SUPERC_09_14,SUPERCPTH09,...,PCH_SNAPS_12_16,SNAPSPTH12,SNAPSPTH16,PCH_SNAPSPTH_12_16,WICS08,WICS12,PCH_WICS_08_12,WICSPTH08,WICSPTH12,PCH_WICSPTH_08_12
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,6,4,-33.333333,0.110834,0.072209,-34.849716,1,1,0.0,0.018472,...,12.694878,0.674004,0.760911,12.894172,6,5,-16.66667,0.119156,0.090067,-24.412460
1003,24,29,20.833333,0.133775,0.144920,8.331001,6,6,0.0,0.033444,...,43.192771,0.725055,0.949753,30.990390,25,27,8.00000,0.141875,0.141517,-0.252126
1005,5,5,0.000000,0.180786,0.185963,2.863838,0,1,NaN,0.000000,...,0.956938,1.280590,1.354387,5.762745,6,7,16.66667,0.201099,0.257344,27.968330
1007,6,5,-16.666667,0.261540,0.222163,-15.055985,1,1,0.0,0.043590,...,20.512821,0.719122,0.864874,20.267995,6,5,-16.66667,0.277919,0.221268,-20.383970
1009,6,6,0.000000,0.104637,0.103952,-0.654897,1,1,0.0,0.017440,...,23.903509,0.657144,0.815946,24.165470,10,6,-40.00000,0.173028,0.103760,-40.033200


In [70]:
# Combine all sheets into one dataframe by joining on FIPS col.

#master_df = []
#for df in dfs:
#    master_df = access.join(df, on="FIPS")
# In [37]: df1.merge(df2, right_index=True, left_on='v2', how='outer')

master_df = access.join(dfs)

In [71]:
access.head()

,FIPS,State,County,LACCESS_POP10,LACCESS_POP15,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,LACCESS_LOWI10,LACCESS_LOWI15,...,LACCESS_HISP15,PCT_LACCESS_HISP15,LACCESS_NHASIAN15,PCT_LACCESS_NHASIAN15,LACCESS_NHNA15,PCT_LACCESS_NHNA15,LACCESS_NHPI15,PCT_LACCESS_NHPI15,LACCESS_MULTIR15,PCT_LACCESS_MULTIR15
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,1001,AL,Autauga,18428.439685,17496.693038,-5.056026,33.769657,32.062255,5344.427472,6543.676824,...,471.136164,0.863345,86.767975,0.159000,61.169869,0.112092,8.817961,0.016159,482.848633,0.884808
1003,1003,AL,Baldwin,35210.814078,30561.264430,-13.204891,19.318473,16.767489,9952.144027,9886.831137,...,1377.874834,0.755973,212.946378,0.116833,181.649648,0.099662,14.819634,0.008131,1127.696098,0.618712
1005,1005,AL,Barbour,5722.305602,6069.523628,6.067799,20.840972,22.105560,3135.676086,2948.790251,...,509.377525,1.855183,17.096410,0.062266,39.960527,0.145539,8.082376,0.029436,462.382655,1.684025
1007,1007,AL,Bibb,1044.867327,969.378841,-7.224696,4.559753,4.230324,491.449066,596.162829,...,8.596762,0.037516,1.994318,0.008703,2.513097,0.010967,0.000000,0.000000,5.259244,0.022951
1009,1009,AL,Blount,1548.175559,3724.428242,140.568857,2.700840,6.497380,609.027708,1650.959482,...,497.489891,0.867886,8.428994,0.014705,28.938242,0.050484,1.062851,0.001854,202.914187,0.353990


In [72]:
master_df['County'].dtype

dtype('O')

In [73]:
master_df.head()

,FIPS,State,County,LACCESS_POP10,LACCESS_POP15,PCH_LACCESS_POP_10_15,PCT_LACCESS_POP10,PCT_LACCESS_POP15,LACCESS_LOWI10,LACCESS_LOWI15,...,METRO13,POPLOSS10,MILK_PRICE10,SODA_PRICE10,MILK_SODA_PRICE10,SODATAX_STORES14,SODATAX_VENDM14,CHIPSTAX_STORES14,CHIPSTAX_VENDM14,FOOD_TAX14
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,1001,AL,Autauga,18428.439685,17496.693038,-5.056026,33.769657,32.062255,5344.427472,6543.676824,...,1,0.0,0.970305,0.972216,0.923289,4.0,4.0,4.0,4.0,4.0
1003,1003,AL,Baldwin,35210.814078,30561.264430,-13.204891,19.318473,16.767489,9952.144027,9886.831137,...,1,0.0,1.017690,1.001391,0.940165,4.0,4.0,4.0,4.0,4.0
1005,1005,AL,Barbour,5722.305602,6069.523628,6.067799,20.840972,22.105560,3135.676086,2948.790251,...,0,0.0,1.136671,0.992584,1.059400,4.0,4.0,4.0,4.0,4.0
1007,1007,AL,Bibb,1044.867327,969.378841,-7.224696,4.559753,4.230324,491.449066,596.162829,...,1,0.0,0.970305,0.972216,0.923289,4.0,4.0,4.0,4.0,4.0
1009,1009,AL,Blount,1548.175559,3724.428242,140.568857,2.700840,6.497380,609.027708,1650.959482,...,1,0.0,0.970305,0.972216,0.923289,4.0,4.0,4.0,4.0,4.0


In [74]:
master_df.shape

(3143, 280)

In [75]:
df = master_df[list(set(features_new))]
df.shape

(3143, 59)

In [76]:
df.tail()

,PCT_FMRKT_OTHERFOOD16,METRO13,PCT_SBP15,PCT_FMRKT_WIC16,MEDHHINC15,CHIPSTAX_VENDM14,PCT_FMRKT_ANMLPROD16,PCT_LACCESS_LOWI15,FSRPTH14,PCT_FMRKT_WICCASH16,...,PCT_LACCESS_MULTIR15,PCT_NSLP15,PCT_FMRKT_SNAP16,FFRPTH14,PCT_SFSP15,State,PCT_FMRKT_CREDIT16,PCT_LACCESS_HISP15,PCT_LACCESS_SNAP15,PCT_HISP10
FIPS,,,,,,,,,,,,,,,,,,,,,
56037,0.000000,0,2.50525,0.0,71867.0,4.0,0.000000,10.845331,0.710953,0.0,...,3.664775,8.847212,0.000000,0.733170,0.739455,WY,0.000000,6.075311,2.141828,15.269598
56039,66.666667,0,2.50525,0.0,83290.0,4.0,66.666667,7.409463,2.398604,0.0,...,2.162273,8.847212,33.333333,1.003053,0.739455,WY,66.666667,3.457192,0.670815,14.985442
56041,0.000000,0,2.50525,0.0,62968.0,4.0,0.000000,9.727151,0.765404,0.0,...,1.658871,8.847212,0.000000,0.765404,0.739455,WY,100.000000,2.536304,2.072485,8.783976
56043,50.000000,0,2.50525,0.0,56088.0,4.0,100.000000,3.621591,1.321798,0.0,...,0.400334,8.847212,0.000000,0.720981,0.739455,WY,50.000000,0.415733,1.053980,13.617719
56045,0.000000,0,2.50525,0.0,60986.0,4.0,0.000000,4.199467,1.527566,0.0,...,0.344217,8.847212,0.000000,0.416609,0.739455,WY,0.000000,0.389472,0.971078,2.996670


In [77]:
df.isnull().sum()

PCT_FMRKT_OTHERFOOD16     895
METRO13                     0
PCT_SBP15                   0
PCT_FMRKT_WIC16           895
MEDHHINC15                  4
CHIPSTAX_VENDM14            0
PCT_FMRKT_ANMLPROD16      895
PCT_LACCESS_LOWI15         20
FSRPTH14                    0
PCT_FMRKT_WICCASH16       895
PCT_LACCESS_NHPI15         19
PCT_FMRKT_SFMNP16         895
PCT_HSPA15               1118
PCT_65OLDER10               0
SODATAX_VENDM14             0
PCT_SNAP16                  0
FOOD_TAX14                  0
PCT_FMRKT_BAKED16         895
PCT_WIC15                   0
SNAPSPTH16                 29
FOODHUB16                   0
CHIPSTAX_STORES14           0
PCT_LACCESS_NHASIAN15      19
FIPS                        0
PCT_LACCESS_POP15          19
County                      0
PCT_LACCESS_SENIORS15      19
PCT_DIABETES_ADULTS13       1
PCT_NHWHITE10               0
PCT_LACCESS_BLACK15        19
RECFACPTH14                 0
GROCPTH14                   0
PCT_LACCESS_HHNV15          3
PCT_LACCES

In [78]:
df = df.dropna()
df.isnull().sum()

PCT_FMRKT_OTHERFOOD16    0
METRO13                  0
PCT_SBP15                0
PCT_FMRKT_WIC16          0
MEDHHINC15               0
CHIPSTAX_VENDM14         0
PCT_FMRKT_ANMLPROD16     0
PCT_LACCESS_LOWI15       0
FSRPTH14                 0
PCT_FMRKT_WICCASH16      0
PCT_LACCESS_NHPI15       0
PCT_FMRKT_SFMNP16        0
PCT_HSPA15               0
PCT_65OLDER10            0
SODATAX_VENDM14          0
PCT_SNAP16               0
FOOD_TAX14               0
PCT_FMRKT_BAKED16        0
PCT_WIC15                0
SNAPSPTH16               0
FOODHUB16                0
CHIPSTAX_STORES14        0
PCT_LACCESS_NHASIAN15    0
FIPS                     0
PCT_LACCESS_POP15        0
County                   0
PCT_LACCESS_SENIORS15    0
PCT_DIABETES_ADULTS13    0
PCT_NHWHITE10            0
PCT_LACCESS_BLACK15      0
RECFACPTH14              0
GROCPTH14                0
PCT_LACCESS_HHNV15       0
PCT_LACCESS_NHNA15       0
PCT_FMRKT_FRVEG16        0
PCT_NHBLACK10            0
PCT_LACCESS_CHILD15      0
P

In [79]:
# make sure metro is boolean
df['METRO13'] = df['METRO13'].astype('bool')

In [80]:
print(df.dtypes)

PCT_FMRKT_OTHERFOOD16    float64
METRO13                     bool
PCT_SBP15                float64
PCT_FMRKT_WIC16          float64
MEDHHINC15               float64
CHIPSTAX_VENDM14         float64
PCT_FMRKT_ANMLPROD16     float64
PCT_LACCESS_LOWI15       float64
FSRPTH14                 float64
PCT_FMRKT_WICCASH16      float64
PCT_LACCESS_NHPI15       float64
PCT_FMRKT_SFMNP16        float64
PCT_HSPA15               float64
PCT_65OLDER10            float64
SODATAX_VENDM14          float64
PCT_SNAP16               float64
FOOD_TAX14               float64
PCT_FMRKT_BAKED16        float64
PCT_WIC15                float64
SNAPSPTH16               float64
FOODHUB16                  int64
CHIPSTAX_STORES14        float64
PCT_LACCESS_NHASIAN15    float64
FIPS                       int64
PCT_LACCESS_POP15        float64
County                    object
PCT_LACCESS_SENIORS15    float64
PCT_DIABETES_ADULTS13    float64
PCT_NHWHITE10            float64
PCT_LACCESS_BLACK15      float64
RECFACPTH1

In [81]:
df.to_csv('data/df_for_webapp.csv', index=False)

In [71]:
df_test = df

In [77]:
df_test['County'] = (df_test['County'].values)

In [78]:
df_test['County']

FIPS
1001         Autauga
1003         Baldwin
1005         Barbour
1007            Bibb
1009          Blount
1011         Bullock
1013          Butler
1015         Calhoun
1017        Chambers
1019        Cherokee
1023         Choctaw
1025          Clarke
1027            Clay
1029        Cleburne
1031          Coffee
1033         Colbert
1035         Conecuh
1039       Covington
1041        Crenshaw
1043         Cullman
1045            Dale
1047          Dallas
1049          DeKalb
1051          Elmore
1053        Escambia
1055          Etowah
1057         Fayette
1059        Franklin
1063          Greene
1065            Hale
            ...     
54093         Tucker
54095          Tyler
54097         Upshur
54099          Wayne
54101        Webster
54103         Wetzel
54107           Wood
54109        Wyoming
56001         Albany
56003       Big Horn
56005       Campbell
56007         Carbon
56009       Converse
56013        Fremont
56015         Goshen
56017    Hot Springs
56019   

In [76]:
df_test.head()

,PCT_LACCESS_NHNA15,PCT_OBESE_ADULTS13,FOOD_TAX14,PCT_FMRKT_OTHERFOOD16,PCT_DIABETES_ADULTS13,PCT_SFSP15,PCT_NHNA10,PCT_HSPA15,SODATAX_STORES14,CHIPSTAX_VENDM14,...,PCT_SNAP16,PCT_NHPI10,MEDHHINC15,County,SPECSPTH14,PCT_NHASIAN10,PCT_FMRKT_SFMNP16,PCT_LACCESS_WHITE15,PCT_LACCESS_POP15,PCT_LACCESS_BLACK15
FIPS,,,,,,,,,,,,,,,,,,,,,
1001,0.112092,34.1,4.0,100.0,13.0,1.027109,0.397647,25.4,4.0,4.0,...,16.976436,0.040314,56580.0,Autauga,0.036104,0.855766,0.0,23.163613,32.062255,7.726582
1003,0.099662,27.4,4.0,75.0,10.4,1.027109,0.628755,25.4,4.0,4.0,...,16.976436,0.043343,52387.0,Baldwin,0.129928,0.735193,100.0,13.981393,16.767489,1.942757
1005,0.145539,44.4,4.0,50.0,18.4,1.027109,0.218524,25.4,4.0,4.0,...,16.976436,0.087409,31433.0,Barbour,0.074385,0.389700,50.0,10.302934,22.105560,9.881360
1007,0.010967,40.3,4.0,0.0,14.8,1.027109,0.279293,25.4,4.0,4.0,...,16.976436,0.030548,40767.0,Bibb,0.044433,0.096007,0.0,2.358770,4.230324,1.828933
1009,0.050484,34.6,4.0,100.0,14.1,1.027109,0.497191,25.4,4.0,4.0,...,16.976436,0.031402,50487.0,Blount,0.000000,0.200621,100.0,5.909147,6.497380,0.167201


In [67]:
print ((df[df['County'] == 'Orange'].County.values))

[u'Orange' u'Orange' u'Orange' u'Orange' u'Orange' u'Orange' u'Orange']


In [84]:
for county in df['County'].values:
    str(county)

df['County'].values

UnicodeEncodeError: 'ascii' codec can't encode character u'\xf1' in position 2: ordinal not in range(128)

In [38]:
df['State'] = df['State'].astype('category')

In [1]:
df.to_csv('data/df_for_webapp.csv', index=False)

NameError: name 'df' is not defined